In [ ]:
import requests
import ollama
from bs4 import BeautifulSoup
from IPython.display import Markdown, display

In [ ]:
MODEL="llama3.2"

In [ ]:
#Defining Sytem prompt
system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."

In [ ]:
#Defining class that represents a website
class Website:
    url: str
    title: str
    text: str

    def __init__(self, url: str):
        self.url = url
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No Title Found"
        for irrelevant in soup.body(["script", "input", "img", "style"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True) 

In [ ]:
#Defining the function that creates user input
def generate_user_prompt_for_website(website: Website) -> str:
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "\nThe contents of this website is as follows;\n\
    please provide a short summary of this website in markdown. \
    If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt

In [ ]:
#function that generates the inout message for the LLM
def generate_messages(website: Website):
    return [
        {
            "role": "system",
            "content": system_prompt
        },
        {  "role": "user",
            "content": generate_user_prompt_for_website(website)
        }
    ]

In [ ]:
def summarize_webpage(url: str):
    webpage = Website(url)
    response = ollama.chat(model=MODEL, messages= generate_messages(webpage))
    return response.message.content

In [ ]:
display(Markdown(summarize_webpage("https://www.aranzulla.it/come-funziona-sito-semplice-di-tim-1272269.html")))